# Using Siamese Network to achieve face recognition
Using AT & T dataset. There are in total 40 peoples, each person has 10 images. The filea are in .pgm format, h=92, w=112.
## 1. Visulization images

In [9]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

files=sorted(os.listdir('../input/att-database-of-faces/s1/'))
image=np.zeros((len(files),128,128,3),dtype=np.uint8)

for i,file in enumerate(files):
    img=cv2.imread(path+file)
    img=cv2.resize(img,(128,128))
    image[i,:,:]=img

plt.figure(figsize=(5,5))
plt.subplot(121)
plt.imshow(image[1,:,:])
plt.subplot(122)
plt.imshow(image[2,:,:])
plt.show()

# Model

In [73]:
import torch
import torch.nn as nn
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
                nn.ReflectionPad2d(1),
                nn.Conv2d(1, 4, kernel_size=3),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(4),
                nn.Dropout2d(p=.2),
                  
                nn.ReflectionPad2d(1),
                nn.Conv2d(4, 8, kernel_size=3),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(8),
                nn.Dropout2d(p=.2),
                      
                nn.ReflectionPad2d(1),
                nn.Conv2d(8, 8, kernel_size=3),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(8),
                nn.Dropout2d(p=.2),
                  )
            
        self.fc1 = nn.Sequential(
                nn.Linear(8*100*100, 500),
                nn.ReLU(inplace=True),
                  
                nn.Linear(500, 500),
                nn.ReLU(inplace=True),
                  
                nn.Linear(500, 5)
                  )

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output
      
    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2
    

# 3. Data Processing

In [4]:
with open('./train.txt','w') as f:
    data_path='../input/att-database-of-faces/'
    for i in range(40):
        for j in range(10):
            img_dir=data_path+'s'+str(i+1)+'/'+str(j+1)+'.pgm'
            f.write(img_dir+' '+str(i)+'\n')
    f.close()

In [22]:
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import linecache
from PIL import Image

train_transform=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])
    
class ATT(Dataset):
    def __init__(self,file,transform):
        self.transform=transform
        self.file=file
    def __len__(self):
        f=open(self.file,'r')
        return len(f.readlines())
    def __getitem__(self,index):
        l=linecache.getline(self.file,np.random.randint(1,self.__len__()))
        img1_dir=l.split()
        should_get_same_class=np.random.randint(0,1)
        if should_get_same_class:
            while True:
                img2_dir=linecache.getline(self.file,np.random.randint(1,self.__len__())).split()
                if img1_dir[1]==img2_dir[1]:
                    break
        else:
            img2_dir=linecache.getline(self.file,np.random.randint(1,self.__len__())).split()
                    
        img1=Image.open(img1_dir[0])
        img1=img1.convert('L')
        img1=self.transform(img1)
        img2=Image.open(img2_dir[0])
        img2=img2.convert('L')
        img2=self.transform(img2)
        return img1,img2,torch.from_numpy(np.array([int(img1_dir[1]!=img2_dir[1])],dtype=np.float32))

In [5]:
train_data=ATT('./train.txt',train_transform)
train_loader=DataLoader(train_data,batch_size=32,shuffle=True)

# 4. define contrastive loss

In [55]:
import torch.nn.functional as F
# Custom Contrastive Loss
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.tensor(torch.mean((1-label) * torch.pow(euclidean_distance, 2) +     # calmp夹断用法
                                      (label)*torch.pow(torch.clamp(self.margin-euclidean_distance,min=0.0),2)),requires_grad=True)
        return loss_contrastive

In [74]:
import numpy as np
import torch.optim as optim
from torch.autograd import Variable
model=Siamese()
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
cri=ContrastiveLoss()
optimizer=optim.Adam(model.parameters(),lr=5e-4)

loss_list=[]
min_loss=10
for epoch in range(50):
    total_loss=0.0
    model.train()
    for i,(img1,img2,target) in enumerate(train_loader):
        #自定义的loss,必须改成Variable或者将loss function改为torch.tensor requires_grad=True，否则无法backward
        img1,img2,target=img1.to(device),img2.to(device),target.to(device)
        optimizer.zero_grad()
        output1,output2=model(img1,img2)
        #print(output1,output2)
        batch_loss=cri(output1,output2,target)
        batch_loss.backward()
        optimizer.step()
        
        total_loss+=batch_loss.item()
    if total_loss<min_loss:
        min_loss=total_loss
        torch.save(model.state_dict(),'./loss{}.pth'.format(min_loss))
    loss_list.append(total_loss)
    print('epoch:',epoch,'loss:',total_loss)
        


# 